# Example usage

Here we will demonstrate how to use `extrucal` to calculate throughput in extrusion and generate the table and plot for throughput with various conditions.

## Imports

In [1]:
from extrucal.extrusion import throughput_cal, throughput_table, throughput_plot
from extrucal.cable_extrusion import req_throughput_cal, req_rpm_table, req_rpm_plot

## 1. Functions in `extrucal.extrusion`

### Calculate throughput in extrusion process

`throughput(size, depth, density, rpm=1, pitch=None, w_flight=None, n_flight=1)` calculates the extrusion throughput (Drag Flow) given the screw size, RPM, the channel depth of metering channel, and screw pitch

In [2]:
throughput_cal(200, 10, 800)

23.51

### Generate a table of throughput in extrusion process

`throughput_table(size, density, pitch=None, w_flight=None, n_flight=1, min_depth=None, max_depth=None, delta_depth=None, min_rpm=5, max_rpm=50, delta_rpm=5)` generates a table containing the extrusion throughput with respect to channel depth and screw RPM

In [3]:
throughput_table(200, 800, max_depth=14) # `max_depth` was used to fit the width of the table

,depth=4.0,depth=6.0,depth=8.0,depth=10.0,depth=12.0,depth=14.0
rpm=5.0,47.94,71.50,94.71,117.54,139.98,162.03
rpm=10.0,95.88,143.00,189.41,235.07,279.96,324.05
rpm=15.0,143.83,214.51,284.12,352.61,419.93,486.08
rpm=20.0,191.77,286.01,378.83,470.14,559.91,648.10
rpm=25.0,239.71,357.51,473.53,587.68,699.89,810.13
rpm=30.0,287.65,429.01,568.24,705.21,839.87,972.15
rpm=35.0,335.60,500.51,662.94,822.75,979.85,1134.18
rpm=40.0,383.54,572.01,757.65,940.28,1119.83,1296.21
rpm=45.0,431.48,643.52,852.36,1057.82,1259.80,1458.23
rpm=50.0,479.42,715.02,947.06,1175.36,1399.78,1620.26


### Generate a plot of throughput in extrusion process

`throughput_plot(size, density, pitch=None, w_flight=None, n_flight=1, min_depth=None, max_depth=None, delta_depth=None, min_rpm=0, max_rpm=50, delta_rpm=1)` generates a plot containing the extrusion throughput with respect to channel depth and screw RPM

In [4]:
throughput_plot(200, 800)

alt.Chart(...)

## 2. Functions in `extrucal.cable_extrusion`

### Calculate the required throughput for cable extrusion

`req_throughput_cal(outer_d, thickness, l_speed, s_density)` calculates the required throughput for given outer diameter, thickness, line speed, and solid density

In [5]:
req_throughput_cal(10, 2, 10, 1000)

30.159

### Generate a table of the required screw RPM in cable extrusion process

`req_rpm_table(outer_d, thickness, s_density, density_ratio=0.85, min_l_speed=1, max_l_speed=10, delta_l_speed=1, min_size=20, max_size=100, delta_size=20, depth_percent=0.05)` generates a table containing the required screw RPM with respect to extruder size and line speed

In [6]:
req_rpm_table(10, 2, 1000, max_l_speed=5) # `max_l_speed` was used to fit the width of the table

,1.0mpm,2.0mpm,3.0mpm,4.0mpm,5.0mpm
20.0mm Ext,150.800000,301.600000,452.400000,603.200000,754.000000
40.0mm Ext,15.080000,30.160000,45.240000,60.320000,75.400000
60.0mm Ext,4.501493,9.002985,13.504478,18.005970,22.507463
80.0mm Ext,1.885000,3.770000,5.655000,7.540000,9.425000
100.0mm Ext,0.966667,1.933333,2.900000,3.866667,4.833333


### Generate a plot of the required screw RPM in cable extrusion process

`req_rpm_plot(outer_d, thickness, s_density, density_ratio=0.85, min_l_speed=1, max_l_speed=10, delta_l_speed=1, min_size=20, max_size=100, delta_size=1, depth_percent=0.05)` generates a plot containing the required screw RPM with respect to extruder size and line speed

In [7]:
req_rpm_plot(10, 2, 1000, min_size=60) # min_size was used to make the plot look better 

alt.Chart(...)